In [ ]:
# !pip install mysql-connector-python
# !pip install nba_api

In [ ]:
import mysql.connector
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
import time

env="prod"
league="NBA"
isPlayoffs = False
getSeason = "22022"

In [ ]:
if league == "NBA":
  league_id = "00"
if league == "WNBA":
  league_id = "10"

In [ ]:
if env == "prod":
  db = mysql.connector.connect(
    host="us-east.connect.psdb.cloud",
    username="",
    password="",
    database=league.lower(),
  )
else:
  db = mysql.connector.connect(
    host="us-east.connect.psdb.cloud",
    username="",
    password="",
    database=league.lower(),
  )
print(db)
cur = db.cursor()

In [ ]:
from nba_api.stats.static import teams

def lookupTeam(abbreviation):
    original = abbreviation
    if abbreviation == "SEA":
        abbreviation = "OKC"
    if abbreviation == "NOH" or abbreviation == "NOK" or abbreviation == "NOR":
        abbreviation = "NOP"
    if abbreviation == "NJN":
        abbreviation = "BKN"
    ret = [team for team in teams.get_teams() if team['abbreviation'] == abbreviation]
    if len(ret) == 0:
        print(original)
    return ret[0]

In [ ]:
from datetime import datetime
now = datetime.now()
print(now)

In [ ]:
def convertMinutesToFloat(minuteString):
  splitted = minuteString.split(":")
  mm = float(splitted[0])
  ss = float(splitted[1])
  return mm + (ss/60)
print(convertMinutesToFloat("10:45"))
print(convertMinutesToFloat("10:30"))
print(convertMinutesToFloat("1:59"))

In [ ]:
# # THIS IS REALLY ONLY REQUIRED THE FIRST TIME THE DB IS INITIALIZED
# get teams
# cur = db.cursor()
# insert_teams_sql = "REPLACE INTO teams (teamID, name, abbreviation, city, league) VALUES (%s, %s, %s, %s, %s;"
# teamValues = []
# for team in teams.get_teams():
#     teamValues.append(
#       (
#         team['id'],
#         team['nickname'],
#         team["abbreviation"],
#         team['city'],
#         league
#       )
#     )
# cur.executemany(insert_teams_sql, teamValues)
# db.commit()
# print("Inserted " + str(cur.rowcount) + " " + league + " Teams!")

In [ ]:
def seasonIDtoSeason(season_id):
  return season_id[1:] + "-"+ str(int(season_id[1:])+1)[2:]

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import boxscoresummaryv2
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscoremiscv2
from nba_api.stats.endpoints import leaguedashptstats

sleeptime = 1

def get_player_game(isPlayoffs, g, boxscore, advancedboxscore, miscboxscore, passingboxscore, teamboxscore, oppboxscore):
  # print(g["PLAYER_NAME"], g["MATCHUP"], g["GAME_DATE"], boxscore["MIN"])
  # print("game: ", g)
  # print("playoffs: ", isPlayoffs)
  # print("boxscore: ", boxscore)
  # print("advanced: ",advancedboxscore)
  # print("misc: ", miscboxscore)
  # print("passing: ", passingboxscore)
  # print("teambox: ", teamboxscore)
  # print("oppbox: ", oppboxscore)
  try:
    outcome = "PENDING"
    if g["WL"] == "W":
      outcome="WIN"
    elif g["WL"] == "L":
      outcome="LOSS"

    season = seasonIDtoSeason(g["SEASON_ID"])

    homeAway="HOME"
    matchup_split = g["MATCHUP"].split(" ")
    if matchup_split[0] == g["TEAM_ABBREVIATION"]:
      if matchup_split[1] == "@":
        homeAway="AWAY"
    else:
      if matchup_split[1] != "@":
        homeAway = "AWAY"

    pot_ast_conv = 0
    potential_assists = passingboxscore["POTENTIAL_AST"]
    if potential_assists > 0:
        pot_ast_conv = (g["AST"]/potential_assists)*100
    
    outcome="PENDING"
    if oppboxscore["PLUS_MINUS"] > 0:
      outcome = "LOSS"
    else:
      outcome="WIN"
    player_game_dict = { 
      "playoffs": isPlayoffs,
      "gameID": g["GAME_ID"],
      "playerID": g["PLAYER_ID"],
      "opponentID": oppboxscore["TEAM_ID"],
      "margin": oppboxscore["PLUS_MINUS"]*-1,
      "teamID": g["TEAM_ID"],
      "outcome": outcome,
      "homeAway": homeAway, 
      "date": datetime.strptime(g["GAME_DATE"], "%Y-%m-%d"), #TODO: Doesn't have time
      "season": season,
      "assists": g["AST"],
      "defensiveRebounds": g["DREB"],
      "offensiveRebounds": g["OREB"],
      "fieldGoalPct": g["FG_PCT"],
      "fieldGoalsAttempted": g["FGA"],
      "fieldGoalsMade": g["FGM"],
      "freeThrowsAttempted": g["FTA"],
      "freeThrowsMade": g["FTM"],
      "freeThrowPct": g["FT_PCT"],
      "points": g["PTS"],
      "threePointPct": g["FG3_PCT"],
      "threePointersAttempted": g["FG3A"],
      "threePointersMade": g["FG3M"],
      "rebounds": g["REB"],
      "turnovers": g["TOV"],
      "blocks": g["BLK"],
      "steals": g["STL"],

      "minutes": convertMinutesToFloat(boxscore["MIN"]),

      "usage": advancedboxscore["USG_PCT"],
      "assistPct": advancedboxscore["AST_PCT"],
      "defensiveReboundPct": advancedboxscore["DREB_PCT"],
      "offensiveReboundPct": advancedboxscore["OREB_PCT"],
      "effectiveFieldGoalPct": advancedboxscore["EFG_PCT"],

      "personalFoulsDrawn": miscboxscore["PFD"],
      "personalFouls": miscboxscore["PF"],

      "potentialAssists":potential_assists,
      "assistConversionRate": pot_ast_conv,
      "passes": passingboxscore["PASSES_MADE"],
    }
    return player_game_dict
  except Exception as e:
    print(e)
    print(g)

In [ ]:
def get_team_game(isPlayoffs, teamgame, teamboxscore, oppboxscore, advanced):
  print("===",teamboxscore["TEAM_NAME"], teamgame["MATCHUP"], teamgame["GAME_DATE"],"===")
  try:
    outcome = "PENDING"
    if teamboxscore["PLUS_MINUS"] > 0:
      outcome="WIN"
    elif teamboxscore["PLUS_MINUS"] < 0:
      outcome="LOSS"

    season = seasonIDtoSeason(teamgame["SEASON_ID"])

    homeAway="HOME"
    matchup_split = teamgame["MATCHUP"].split(" ")
    if matchup_split[0] == teamboxscore["TEAM_ABBREVIATION"]:
      if matchup_split[1] == "@":
        homeAway="AWAY"
    else:
      if matchup_split[1] != "@":
        homeAway = "AWAY"

    team_game_data = {
        "gameID": teamboxscore["GAME_ID"],
        "date": datetime.strptime(teamgame["GAME_DATE"], "%Y-%m-%d"),
        "margin": teamboxscore["PLUS_MINUS"],
        "points": teamboxscore["PTS"],
        "assists": teamboxscore["AST"],
        "rebounds": teamboxscore["REB"],
        "offensiveRebounds": teamboxscore["OREB"],
        "defensiveRebounds": teamboxscore["DREB"],
        "turnovers": teamboxscore["TO"],
        "steals": teamboxscore["STL"],
        "blocks": teamboxscore["BLK"],
        "threePointersAttempted": teamboxscore["FG3A"],
        "threePointersMade": teamboxscore["FG3M"],
        # "defensiveRating": advanced["DEF_RATING"],
        "effectiveFieldGoalPct": advanced["EFG_PCT"],
        "defensiveReboundPct": advanced["DREB_PCT"],
        "fieldGoalPct": teamboxscore["FG_PCT"],
        "fieldGoalsAttempted": teamboxscore["FGA"],
        "fieldGoalsMade": teamboxscore["FGM"],
        "homeAway":homeAway,
        "offensiveReboundPct": advanced["OREB_PCT"],
        "opponentID":oppboxscore["TEAM_ID"],
        "personalFouls": teamboxscore["PF"],
        "plusMinusPerHundred": ((int(teamboxscore["PTS"])-int(oppboxscore["PTS"]))/float(advanced["POSS"]))*100,
        "possessions": advanced["POSS"],
        "season": season,
        "teamID": teamboxscore["TEAM_ID"],
        "outcome":outcome,
        "playoffs": isPlayoffs,
    }
    return team_game_data
  except Exception as e:
      print("ERROR: ", e)
      print("GAME FOR REFERENCE: ", teamboxscore)

In [ ]:
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import commonplayerinfo

def get_player_data(playerID):
  time.sleep(sleeptime)
  player = c = commonplayerinfo.CommonPlayerInfo(player_id=playerID).get_normalized_dict()["CommonPlayerInfo"][0]
  ftInches = player["HEIGHT"].split("-")
  inches = int(ftInches[0])*12+int(ftInches[1])
  position = "-".join([position[0] for position in player["POSITION"].split("-",2)])
  player_data = {
        "playerID":	player["PERSON_ID"],
        "firstName":player["FIRST_NAME"],
        "lastName": player["LAST_NAME"],
        "name": player["DISPLAY_FIRST_LAST"],
        "height":player["HEIGHT"],
        "weight":player["WEIGHT"],
        "heightInches":inches,
        "position":position,
        "league":league,
        "teamID":player["TEAM_ID"],
        "teamABR":player["TEAM_ABBREVIATION"],
  }
  return player_data

In [ ]:
replace_into_table = """REPLACE INTO {table} ({columns}) VALUES ({values})"""

def get_stmt(t, items):
  return replace_into_table.format(table=t, columns=",".join(items[0].keys()), values=", ".join(["%s"] * len(items[0])))

processedGames = []

In [ ]:
#REGULAR SEASON
time.sleep(sleeptime)
if isPlayoffs:
  season_types = ["Playoffs", "PlayIn"]
season_types = ["Regular Season"]
attempt = 0
debug=False
cur = db.cursor()
for attempt in range(5):
    print("attempt: " + str(attempt))
    try:
      for season_type in season_types:  
        if debug:
          print("GET PLAYER GAMES")
        playergamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable=league_id, season_type_nullable=season_type, player_or_team_abbreviation="P").get_normalized_dict()["LeagueGameFinderResults"]
        
        if debug:
          print("GET TEAM GAMES")
        time.sleep(sleeptime)
        teamgamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable=league_id, season_type_nullable=season_type, player_or_team_abbreviation="T").get_normalized_dict()["LeagueGameFinderResults"]
        time.sleep(sleeptime)
        
        if debug:
          print("GET FIRST BOX SCORES")
        firstID = playergamefinder[0]["GAME_ID"]
        useDate = playergamefinder[0]["GAME_DATE"]
        boxscorefinder = boxscoretraditionalv2.BoxScoreTraditionalV2(firstID).get_normalized_dict()
        
        if debug:
          print("GET FIRST ADVANCED SCORES")
        time.sleep(sleeptime)
        advancedfinder = boxscoreadvancedv2.BoxScoreAdvancedV2(firstID).get_normalized_dict()
        
        if debug:
          print("GET FIRST MISC SCORES")
        time.sleep(sleeptime)
        miscfinder = boxscoremiscv2.BoxScoreMiscV2(firstID).get_normalized_dict()
        
        if debug:
          print("GET FIRST PASSING SCORES")
          print(getSeason, seasonIDtoSeason(getSeason))
        time.sleep(sleeptime)
        players_single_day_passing = leaguedashptstats.LeagueDashPtStats(pt_measure_type="Passing", season=seasonIDtoSeason(getSeason), date_from_nullable=useDate, date_to_nullable=useDate, player_or_team="Player").get_normalized_dict()["LeagueDashPtStats"]
        time.sleep(sleeptime)
        playergames = []
        teamgames = []
        for teamgame in [teamgame for teamgame in teamgamefinder if teamgame["SEASON_ID"] == getSeason and teamgame["WL"] != None]:
          if teamgame["GAME_ID"] in processedGames:
            continue
          # TODO: Review what happens if a game is ongoing...
          if useDate != teamgame["GAME_DATE"]:
            if len(teamgames) > 0:
              update_teamgames_stmt = get_stmt("teamgames", teamgames)
              # upload teamgames
              cur.executemany(update_teamgames_stmt, [list(t.values()) for t in teamgames])
              db.commit()
              print("UPLOADED ", len(teamgames), " TEAM GAMES")
            if len(playergames) > 0:
              cur = db.cursor()
              # upload playergames
              uopdate_playergames_stmt = get_stmt("playergames", playergames)
              cur.executemany(uopdate_playergames_stmt, [list(p.values()) for p in playergames])
              print("UPLOADED ", len(playergames), " PLAYER GAMES")
              playerIDs = [game["playerID"] for game in playergames]
              if debug:
                print("GETTING PLAYERS FROM DB")
              cur.execute("SELECT playerID FROM players WHERE playerID IN (%s)" % ",".join(['%s'] * len(playerIDs)), playerIDs)
              received = cur.fetchall()
              received = [r[0] for r in received]
              missingPlayerIDs = [player for player in playerIDs if player not in received]
              if debug:
                print("searched for ", len(playerIDs), "found ", len(received), ", ",len(missingPlayerIDs), " of which were missing players")
              if len(missingPlayerIDs) > 0:
                missingPlayers = []
                for missingID in missingPlayerIDs:
                  player_data = get_player_data(missingID)
                  missingPlayers.append(player_data)
                cur = db.cursor()
                update_players_stmt = get_stmt("players", missingPlayers)
                cur.executemany(update_players_stmt, [list(p.values()) for p in missingPlayers])
                db.commit()
                print("UPLOADED ", len(missingPlayers), " MISSING PLAYERS: ", [player["name"] for player in missingPlayers])
              print()
              print()
              cur = db.cursor()
              playergames = []
              teamgames = []
            useDate = teamgame["GAME_DATE"]
            if debug:
                print("Getting passing boxscore again")
                print(getSeason, seasonIDtoSeason(getSeason))
            players_single_day_passing = leaguedashptstats.LeagueDashPtStats(pt_measure_type="Passing",date_from_nullable=useDate, season=seasonIDtoSeason(getSeason), date_to_nullable=useDate, player_or_team="Player").get_normalized_dict()["LeagueDashPtStats"]
            time.sleep(sleeptime)
          if boxscorefinder["TeamStats"][0]["GAME_ID"] != teamgame["GAME_ID"]:
            boxscorefinder = boxscoretraditionalv2.BoxScoreTraditionalV2(teamgame["GAME_ID"]).get_normalized_dict()
            time.sleep(sleeptime)
          if advancedfinder["TeamStats"][0]["GAME_ID"] != teamgame["GAME_ID"]:
            advancedfinder = boxscoreadvancedv2.BoxScoreAdvancedV2(teamgame["GAME_ID"]).get_normalized_dict()
            time.sleep(sleeptime)
          if miscfinder["sqlTeamsMisc"][0]["GAME_ID"] != teamgame["GAME_ID"]:
            miscfinder = boxscoremiscv2.BoxScoreMiscV2(teamgame["GAME_ID"]).get_normalized_dict()
            time.sleep(sleeptime)
          
          if debug:
            print("Getting team boxscore")
          teamboxscores = [boxscore for boxscore in boxscorefinder["TeamStats"] if boxscore["TEAM_ID"] == teamgame["TEAM_ID"]]
          if len(teamboxscores) == 0:
            raise Exception ("teamboxscore not found")
          teamboxscore = teamboxscores[0]
          
          if debug:
            print("Getting opponent boxscore")
          opponentboxscores = [boxscore for boxscore in boxscorefinder["TeamStats"] if boxscore["TEAM_ID"] != teamgame["TEAM_ID"]]
          if len(opponentboxscores) == 0:
            raise Exception ("opponentboxscore not found")
          opponentboxscore = opponentboxscores[0]
          
          if debug:
            print("Getting advanced team boxscore")
          advancedteamboxscores = [boxscore for boxscore in advancedfinder["TeamStats"] if boxscore["TEAM_ID"] == teamgame["TEAM_ID"]]
          if len(advancedteamboxscores) == 0:
            raise Exception ("advancedteamboxscore not found")
          advancedteamboxscore = advancedteamboxscores[0]
          
          if debug:
            print("Getting advanced opponent boxscore")
          advancedopponentboxscores = [boxscore for boxscore in advancedfinder["TeamStats"] if boxscore["TEAM_ID"] != teamgame["TEAM_ID"]]
          if len(advancedopponentboxscores) == 0:
            raise Exception ("advancedopponentboxscore not found")
          advancedopponentboxscore = advancedopponentboxscores[0]

          # get team (both) game data from box scores
          tg = get_team_game(isPlayoffs, teamgame, teamboxscore, opponentboxscore, advancedteamboxscore)
          oppg = get_team_game(isPlayoffs, teamgame, opponentboxscore, teamboxscore, advancedopponentboxscore)
          teamgames.append(tg)
          teamgames.append(oppg)

          for game in [pgame for pgame in playergamefinder if pgame["GAME_ID"] == teamgame["GAME_ID"] and pgame["MIN"] > 0]:
            try:
              
              if debug:
                print("Getting player boxscore")
              playerboxscores = [boxscore for boxscore in boxscorefinder["PlayerStats"] if boxscore["PLAYER_ID"] == game["PLAYER_ID"]]
              if len(playerboxscores) == 0:
                raise Exception ("playerboxscore not found")
              playerboxscore = playerboxscores[0]
              
              if debug:
                print("Getting advanced boxscore")
              advancedboxscores = [box for box in advancedfinder["PlayerStats"] if box["PLAYER_ID"] == game["PLAYER_ID"]]
              if len(advancedboxscores) == 0:
                raise Exception ("advancedboxscore not found")
              advancedboxscore = advancedboxscores[0]
              
              if debug:
                print("Getting misc boxscore")
              miscboxscores = [box for box in miscfinder["sqlPlayersMisc"] if box["PLAYER_ID"] == game["PLAYER_ID"]]
              if len(miscboxscores) == 0:
                raise Exception ("miscboxscore not found")
              miscboxscore = miscboxscores[0]
              
              if debug:
                print("Getting passing boxscore")
                print(getSeason, seasonIDtoSeason(getSeason))
              passingboxscores = [box for box in players_single_day_passing if box["PLAYER_ID"] == game["PLAYER_ID"]]
              if len(passingboxscores) == 0:
                raise Exception ("passingboxscore not found")
              passingboxscore = passingboxscores[0]
              
              if debug:
                print("Getting team boxscore (2)")
              teamboxscores = [boxscore for boxscore in boxscorefinder["TeamStats"] if boxscore["TEAM_ID"] == game["TEAM_ID"]]
              if len(teamboxscores) == 0:
                raise Exception("teamboxscore not found")
              teamboxscore = teamboxscores[0]
              
              if debug:
                print("Getting opponent boxscore (2)")
              opponentboxscores = [boxscore for boxscore in boxscorefinder["TeamStats"] if boxscore["TEAM_ID"] != game["TEAM_ID"]]
              if len(opponentboxscores) == 0:
                raise Exception("opponentboxscore not found")
              opponentboxscore = opponentboxscores[0]

              # get player game data from boxscores
              pg = get_player_game(isPlayoffs, game, playerboxscore, advancedboxscore, miscboxscore, passingboxscore, teamboxscore, opponentboxscore)
              playergames.append(pg)
              if len(playergames) >= 5:
                cur = db.cursor()
                # upload playergames
                update_players_stmt = replace_into_table.format(table="playergames", columns=",".join(playergames[0].keys()), values=", ".join(["%s"] * len(playergames[0])))
                cur.executemany(update_players_stmt, [list(p.values()) for p in playergames])
                db.commit()
                print("UPLOADED ", len(playergames), " PLAYER GAMES")
                playerIDs = [game["playerID"] for game in playergames]
                if debug:
                  print("GETTING PLAYERS FROM DB")
                cur.execute("SELECT playerID FROM players WHERE playerID IN (%s)" % ",".join(['%s'] * len(playerIDs)), playerIDs)
                received = cur.fetchall()
                received = [r[0] for r in received]
                missingPlayerIDs = [player for player in playerIDs if player not in received]
                if debug:
                  print("searched for ", len(playerIDs), "found ", len(received), ", ",len(missingPlayerIDs), " of which were missing players")
                if len(missingPlayerIDs) > 0:
                  missingPlayers = []
                  for missingID in missingPlayerIDs:
                    player_data = get_player_data(missingID)
                    missingPlayers.append(player_data)
                  cur = db.cursor()
                  update_players_stmt = get_stmt("players", missingPlayers)
                  cur.executemany(update_players_stmt, [list(p.values()) for p in missingPlayers])
                  db.commit()
                  print("UPLOADED ", len(missingPlayers), " MISSING PLAYERS: ", [player["name"] for player in missingPlayers])
                cur = db.cursor()
                playergames = []
            except Exception as e:
              print(e)
              print("Failed to get player game:", game)
          print()
          print()
          processedGames.append(teamgame["GAME_ID"])
        break
    except Exception as e:
        print("failed on attempt: " + str(attempt))
        print(e)
        # print(game)
        time.sleep(sleeptime*10)